# Fuzzy upravljanje poljoprivrednom proizvodnjom u plasteniku

<center><img src="Dinko.png" alt="drawing" width="50%"/></center>

### KRATAK OPIS PROBLEMA 
<p>
 Kao što nam je poznato u plastenicima i staklenicima se vrlo brzo mijenjaju klimatski uvjeti
 i zbog toga nam je potrebna automatska regulacija ventilacije (temperature) kako bi se potpuno iskoristila funkcionalnost objekata.

Zbog previsoke temperature u objektu dolazi do izduživanja internodija na biljkama dok preniska 
temperatura zaustavlja njezin rast. Kod ručnog otvaranja ventilacije nikad ne možemo pravovremeno 
odrediti kada je potrebno otvaranje ili zatvaranje ventilacije, a već mala zakašnjenja mogu dovesti do
 ekstremno visokih temperatura što je veliki šok za biljke.

Sa automatikom se ventilacija vrši postepenim otvaranjem prozora čime se onemogućavaju nagle 
promijene temperature koje uzrokuju šokiranje biljaka. Kod određene brzine i smjera vjetra automatika 
po potrebi pritvara ili zatvara ventilacijske otvore kako bi se spriječilo oštećenje objekta kod nevrjemena.<sup>[1](#izvori)</sup>
</p>

### DEFINICIJA ULAZNIH I IZLAZNIH VARIJABLI

Projekat bi bio realiziran na platformi Raspberry pi pico w, zbog svoje podrške za Python programski jezik i zato što ima ugrađen WiFi čip na sebi koji ćemo iskoristiti u projektu. U nastavku su definisane ulazne i izlazne varijable koje ćemo koristiti za mikrokontroler:

#### Ulazne varijable

Za naš fuzzy kontroler smo koristili 5 ulaznih varijabli:

1. **Varijabla:** Temperatura zraka  
   **Senzor:** `DHT 22`

2. **Varijabla:** Vlažnost zraka  
   **Senzor:** `DHT 22`

3. **Varijabla:** Brzina vjetra  
   **Senzor:** `Adafruit anemometar`

4. **Varijabla:** AQ indeks  
   **API:** [dajposlic.pythonanywhere.com](https://dajposlic.pythonanywhere.com) (što je omotač za [AQICN API](https://aqicn.org/city/sarajevo/))

5. **Varijabla:** Da li pada kiša  
   **Senzor:** `Senzor za kišu`


#### Izlazne varijable

Izlazna varijabla je tipa **bool** i predstavlja signal mikrokontroleru da treba otvoriti prozore.

> ili

Izlazna varijabla je tipa **double** i predstavlja koliko će mikrokontroler otvoriti prozore.




### DEFINISANJE FUZZY SKUPOVA

1. **Varijabla:** Temperatura zraka  
   **Tip varijable:** `kontinualna`  
   **Domen:** [-40°C, 80°C]<sup>[2](#izvori)</sup>  
   **Lingvistički model varijable:**  
   - `hladno`  
   - `toplo`  
   - `vruće`


2. **Varijabla:** Vlažnost zraka  
   **Tip varijable:** `kontinualna`  
   **Domen:** [0%, 100%]  
   **Lingvistički model varijable:**  
   - `vlažno`  
   - `suho`


3. **Varijabla:** Brzina vjetra  
   **Tip varijable:** `kontinualna`  
   **Domen:** [0 m/s, 32.4 m/s]  
   **Lingvistički model varijable:**  
   - `nema`  
   - `slab`  
   - `umjeren`  
   - `jak`

4. **Varijabla:** AQ indeks  
   **Tip varijable:** `kontinualna`  
   **Domen:** [0, +∞]  
   **Lingvistički model varijable:**<sup>[3](#izvori)</sup>
   - `dobro`  
   - `umjereno`  
   - `blago nezdravo`  
   - `nezdravo`  
   - `pravo nezdravo`  
   - `opasno`

5. **Varijabla:** Da li pada kiša  
   **Tip varijable:** `diskretna`  
   **Domen:** {0, 1}  
   **Lingvistički model varijable:**  
   - `ne pada kiša`  
   - `pada kiša`


### IMPLEMENTACIJA FUZZY SISTEMA

### ZAKLJUČAK I KRATKA REFLEKSIJA

### IZVORI

1. https://www.winter.hr/automatika.htm
1. https://cdn.sparkfun.com/assets/f/7/d/9/c/DHT22.pdf
1. https://waqi.info/#/c/3.396/8.206/2.2z



### ULOGE ČLANOVA